In [1]:
### IMPORTING LIBRARIES

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import preprocessing


from sklearn import metrics
from sklearn.metrics import jaccard_similarity_score as jscore
from sklearn.metrics import accuracy_score as acc


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict


from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [5]:
## CHANGING THE NAME OF COLUMNS:

columns=['ID', 'gender', 'married', 'dependents', 'edu',
       'employment', 'income', 'coincome', 'loan',
       'term', 'credit', 'area', 'status']

columns1=['ID', 'gender', 'married', 'dependents', 'edu',
       'employment', 'income', 'coincome', 'loan',
       'term', 'credit', 'area']


train.columns=columns
test.columns=columns1


In [6]:
train.describe(include='all')

,ID,gender,married,dependents,edu,employment,income,coincome,loan,term,credit,area,status
count,614,601,611,599,614,582,614.000000,614.000000,592.000000,600.00000,564.000000,614,614
unique,614,2,2,4,2,2,NaN,NaN,NaN,NaN,NaN,3,2
top,LP002201,Male,Yes,0,Graduate,No,NaN,NaN,NaN,NaN,NaN,Semiurban,Y
freq,1,489,398,345,480,500,NaN,NaN,NaN,NaN,NaN,233,422
mean,NaN,NaN,NaN,NaN,NaN,NaN,5403.459283,1621.245798,146.412162,342.00000,0.842199,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,6109.041673,2926.248369,85.587325,65.12041,0.364878,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,150.000000,0.000000,9.000000,12.00000,0.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,2877.500000,0.000000,100.000000,360.00000,1.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,3812.500000,1188.500000,128.000000,360.00000,1.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,5795.000000,2297.250000,168.000000,360.00000,1.000000,NaN,NaN


In [7]:
train.isnull().sum()
### 'credit' and 'employment' has maximum missing values 

ID             0
gender        13
married        3
dependents    15
edu            0
employment    32
income         0
coincome       0
loan          22
term          14
credit        50
area           0
status         0
dtype: int64

In [8]:
### REMOVING ROWS IN WHICH LOAN AMOUNT IS NOT MENTIONED:

train.dropna(subset=['loan'], inplace=True)

In [9]:
train.credit.value_counts()

1.0    458
0.0     85
Name: credit, dtype: int64

In [10]:
train.employment.value_counts()

No     482
Yes     79
Name: employment, dtype: int64

In [11]:
train.gender.value_counts()

Male      470
Female    109
Name: gender, dtype: int64

In [12]:
train.married .value_counts()

Yes    384
No     206
Name: married, dtype: int64

In [13]:
train.dependents .value_counts()

0     334
1      98
2      98
3+     49
Name: dependents, dtype: int64

In [14]:
train.married .value_counts()

Yes    384
No     206
Name: married, dtype: int64

In [15]:
train[['employment','credit']].groupby(['employment']).mean()

,credit
employment,
No,0.835955
Yes,0.847222


In [16]:
train[['married','credit']].groupby(['married']).mean()

,credit
married,
No,0.839378
Yes,0.844828


In [17]:
train[['dependents','credit']].groupby(['dependents']).mean()

,credit
dependents,
0,0.856209
1,0.848837
2,0.858696
3+,0.782609


In [18]:
train[['gender','credit']].groupby(['gender']).mean()

,credit
gender,
Female,0.836735
Male,0.847575


In [19]:
train[['edu','credit']].groupby(['edu']).mean()

,credit
edu,
Graduate,0.857809
Not Graduate,0.789474


In [20]:
train[['area','credit']].groupby(['area']).mean()

,credit
area,
Rural,0.836478
Semiurban,0.858491
Urban,0.831395


In [21]:
train[['status','credit']].groupby(['status']).mean()

,credit
status,
N,0.538462
Y,0.981283


In [22]:
### FILLING '0' VALUES IN CREDIT IF STATUS IS 'NO' :

a=train[(train.status=='N')]

a['credit'].fillna(value=0.0, inplace=True)

train[(train.status=='N')] = a[(a.status=='N')]

train.credit.fillna(value=1.0, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4535: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [23]:
train.isnull().sum()

ID             0
gender        13
married        2
dependents    13
edu            0
employment    31
income         0
coincome       0
loan           0
term          14
credit         0
area           0
status         0
dtype: int64

In [24]:
### FILLING THE REMAINING MISSING IN ALL COLUMNS:

train.employment.fillna(value='No', inplace=True)
train.gender.fillna(value='Male', inplace=True)
train.married.fillna(value='No', inplace=True)
train.dependents.fillna(value='0', inplace=True)
train.term.fillna(value=360, inplace=True)



In [25]:
train.isnull().sum()


ID            0
gender        0
married       0
dependents    0
edu           0
employment    0
income        0
coincome      0
loan          0
term          0
credit        0
area          0
status        0
dtype: int64

In [26]:
### NOW FILLING MISSING VALUES FOR TEST DATASET:

test.isnull().sum()

ID             0
gender        11
married        0
dependents    10
edu            0
employment    23
income         0
coincome       0
loan           5
term           6
credit        29
area           0
dtype: int64

In [27]:
### FILLING VALUES OF TEST DATASET:

test.employment.fillna(value='No', inplace=True)
test.gender.fillna(value='Male', inplace=True)
test.married.fillna(value='No', inplace=True)
test.dependents.fillna(value='0', inplace=True)
test.term.fillna(value=360, inplace=True)
test.loan.fillna(value=test['loan'].mean(), inplace=True)
test.credit.fillna(value=1.0, inplace=True)

In [28]:
train.head()

,ID,gender,married,dependents,edu,employment,income,coincome,loan,term,credit,area,status
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


In [29]:
test.head()

,ID,gender,married,dependents,edu,employment,income,coincome,loan,term,credit,area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,1.0,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [30]:
train.status.replace('N',0,inplace=True)
train.status.replace('Y',1,inplace=True)
train.corr()

,income,coincome,loan,term,credit,status
income,1.000000,-0.113695,0.570909,-0.048588,-0.013157,-0.004514
coincome,-0.113695,1.000000,0.188619,-0.059346,-0.060746,-0.065034
loan,0.570909,0.188619,1.000000,0.036981,-0.024176,-0.037318
term,-0.048588,-0.059346,0.036981,1.000000,-0.000772,-0.031256
credit,-0.013157,-0.060746,-0.024176,-0.000772,1.000000,0.597722
status,-0.004514,-0.065034,-0.037318,-0.031256,0.597722,1.000000


In [31]:
### PREPARING X AND X_TEST:

y=train['status'].values

X=train[['gender', 'married', 'dependents', 'edu','employment','income','coincome','loan','term','credit','area']].values
X_test=test[['gender', 'married', 'dependents', 'edu','employment','income','coincome','loan','term','credit','area']].values



In [32]:
### LABEL ENCODING:

le_gender=preprocessing.LabelEncoder()
le_married=preprocessing.LabelEncoder()
le_dependents=preprocessing.LabelEncoder()
le_edu=preprocessing.LabelEncoder()
le_employment=preprocessing.LabelEncoder()
le_area=preprocessing.LabelEncoder()


## FOR GENDER

le_gender.fit(['Male','Female'])

X[:,0]=le_gender.transform(X[:,0])

X_test[:,0]=le_gender.transform(X_test[:,0])

## FOR MARRIED

le_married.fit(['Yes','No'])

X[:,1]=le_married.transform(X[:,1])

X_test[:,1]=le_married.transform(X_test[:,1])

## FOR DEPENDENTS

le_dependents.fit(['0','1','2','3+'])

X[:,2]=le_dependents.transform(X[:,2])

X_test[:,2]=le_dependents.transform(X_test[:,2])

## FOR EDUCATION

le_edu.fit(['Not Graduate','Graduate'])

X[:,3]=le_edu.transform(X[:,3])

X_test[:,3]=le_edu.transform(X_test[:,3])

## FOR EMPLOYMENT

le_employment.fit(['Yes','No'])

X[:,4]=le_employment.transform(X[:,4])

X_test[:,4]=le_employment.transform(X_test[:,4])

## FOR AREA

le_area.fit(['Rural','Semiurban','Urban'])

X[:,10]=le_area.transform(X[:,10])

X_test[:,10]=le_area.transform(X_test[:,10])


In [33]:
###ONE HOT ENCODING:(FOR GENDER)

from sklearn.preprocessing import OneHotEncoder


encoder = OneHotEncoder()

A=encoder.fit_transform(X[:,0].reshape(-1,1))

A=A.toarray()

X=np.concatenate((X,A),axis=1)
X=np.delete(X,[0],axis=1)


A=encoder.fit_transform(X_test[:,0].reshape(-1,1))

A=A.toarray()

X_test=np.concatenate((X_test,A),axis=1)
X_test=np.delete(X_test,[0],axis=1)

In [34]:
X_=X.copy()
X_test_=X_test.copy()

In [35]:
### STANDARDISING X AND X_TEST:


X_=preprocessing.StandardScaler().fit(X).transform(X)
X_test_=preprocessing.StandardScaler().fit(X_test).transform(X_test)

In [36]:
###TEST TRAIN SPLIT:

X_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)
x_train,x_test_,Y_train,Y_test=train_test_split(X_,y,test_size=0.3,random_state=0) 

In [37]:
### LOGISTIC REGRESSION:(WITH STANDARDISED FEATURES)

from sklearn.linear_model import LogisticRegression

lr=LogisticRegression(C=5, solver='liblinear').fit(x_train,Y_train)

yhat=lr.predict(x_test_)

acc(Y_test,yhat)*100



83.14606741573034

In [38]:
### CROSS VALIDATION

scores=cross_val_score(lr,X_train,y_train,cv=5)

yhat0=cross_val_predict(lr,X_train,y_train,cv=5)

np.mean(scores)*100

83.08845136644138

In [39]:
# KNN:(WITHOUT STANDARDISED FEATURES)

from sklearn.neighbors import KNeighborsClassifier

k=300

neigh=KNeighborsClassifier(n_neighbors=k).fit(x_train,Y_train)

yhat1=neigh.predict(x_test_)

acc(Y_test,yhat1)*100

68.53932584269663

In [40]:
from sklearn import svm

clf1=svm.SVC(kernel='linear', C=0.01)

clf1.fit(x_train,Y_train)

yhat3=clf1.predict(x_test_)

(acc(Y_test,yhat3))*100


83.14606741573034

In [41]:
### SVM (POLYNOMIAL)(STANDARDISED FEATURES)

from sklearn import svm

clf2=svm.SVC(kernel='poly',degree=2,coef0=1, C=5)

clf2.fit(x_train,Y_train)

yhat3=clf2.predict(x_test_)

(acc(Y_test,yhat3))*100


79.21348314606742

In [42]:
### SVM (RBF)(STANDARDISED FEATURES)

from sklearn import svm

clf3=svm.SVC(kernel='rbf',gamma=0.01, C=4)


clf3.fit(x_train,Y_train)

yhat3=clf3.predict(x_test_)

(acc(Y_test,yhat3))*100


83.14606741573034

In [43]:
### DECISION TREE:(WITHOUT STANDARDISED FEATURES)

from sklearn.tree import DecisionTreeClassifier

tree=DecisionTreeClassifier(criterion='entropy', max_depth=3)



tree.fit(x_train,Y_train)

yhat2=tree.predict(x_test_)

acc(Y_test,yhat2)*100

83.14606741573034

In [44]:
### RANDOM FOREST CLASSIFIER

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

# Choose the type of classifier. 
rnd = RandomForestClassifier()


### TO GET PARAMETERS OF ANY MODEL:

rnd.get_params().keys()



dict_keys(['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [45]:
### GRIDSEARCH + RANDOM FOREST CLASSIFIER:


# Choose some parameter combinations to try
parameters = {'n_estimators': [4, 6, 9], 
              'max_features': ['log2', 'sqrt','auto'], 
              'criterion': ['entropy', 'gini'],
              'max_depth': [2, 3,5, 6, 10], 
              'min_samples_split': [2, 3, 5],
              'min_samples_leaf': [1,5,8]
             }

# Type of scoring used to compare parameter combinations
acc_scorer = make_scorer(accuracy_score)

# Run the grid search
grid_obj = GridSearchCV(rnd, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(x_train, Y_train)

# Set the clf to the best combination of parameters
rnd = grid_obj.best_estimator_  

# Fit the best algorithm to the data. 
rnd.fit(x_train, Y_train)

yhat7=rnd.predict(x_test_)

(acc(Y_test,yhat7))*100


83.14606741573034

In [46]:
### VOTING CLASSIFIER(ENSEMBLE)(WITHOUT STANDARDISED FEATURES)

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC

lr=LogisticRegression(C=6, solver='liblinear')
##rnd=RandomForestClassifier()
svm=SVC(kernel='poly',degree=5,coef0=1, C=6)
dt=DecisionTreeClassifier(criterion='entropy', max_depth=3)
knn=KNeighborsClassifier(n_neighbors=k)
rbf=SVC(kernel='rbf',gamma=0.01, C=8)

voting_clf=VotingClassifier(estimators=[('lr',lr),('rf', rnd),('dt',dt),('knn',knn),('rbf',rbf)], voting='hard')

voting_clf.fit(x_train,Y_train)

yhat6=voting_clf.predict(x_test_)

(acc(Y_test,yhat6))*100


83.14606741573034

In [47]:
### TRAINING ON FULL GIVEN DATASET

# WITH Logistic Regression:

lr.fit(X_,y)


yhat_test=lr.predict(X_test_)

yhat_test

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,

In [48]:
### PREPARING FOR KAGGLE SUBMISSION FORMAT

df = pd.DataFrame(yhat_test)
df.columns=['Loan_Status']

## CONVERTING 0,1 INTO Y,N:


df.Loan_Status.replace(0,'N',inplace=True)
df.Loan_Status.replace(1,'Y',inplace=True)
df.head()

,Loan_Status
0,Y
1,Y
2,Y
3,Y
4,Y


In [49]:
## STORING IN prediction.csv

pred = pd.concat([test, df], axis = 1)

pred.columns=['Loan_ID', 'gender', 'married', 'dependents', 'edu', 'employment', 'income','coincome', 'loan', 'term', 'credit', 'area', 'Loan_Status']

pred.index=pred.Loan_ID
pred=pred[['Loan_Status']]


pred.to_csv('prediction.csv')

